# Setup

In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import h5py
import random
import datetime
import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from numpy.linalg import cond
from typing import List

# Matrixlib

import matrixlib.block as blk
from matrixlib import preconditioning as prec
from matrixlib.core import MatrixData, ValueProperties, BlockProperties

# Modellib
import modellib.gcn
import modellib.io
import modellib.train
import modellib.evaluate 
import modellib.losses

In [5]:
# Def constants and variables
NUM_NODES = 64
NUM_FEATURES = 21

class_weights = {0: 0.2, 1: 0.8}

# Load Data

In [9]:
importlib.reload(modellib.io)

# Load data
train_bands, train_labels = modellib.io.load_from_hdf5('train.h5')
val_bands, val_labels = modellib.io.load_from_hdf5('val.h5')
test_bands, test_labels = modellib.io.load_from_hdf5('test.h5')

# Reshape
train_reshaped = np.squeeze(train_bands).transpose(0, 2, 1)
val_reshaped = np.squeeze(val_bands).transpose(0, 2, 1)
test_reshaped = np.squeeze(test_bands).transpose(0, 2, 1)

# Create initial adj & normalize
adj_matrix = modellib.gcn.create_initial_adj_matrix(NUM_NODES)
adj_matrix = adj_matrix / np.sum(adj_matrix, axis=1, keepdims=True)

# Repeat adj_matrix for each sample in train, val, and test sets
adj_matrix_train = np.repeat(adj_matrix[np.newaxis, :, :], train_reshaped.shape[0], axis=0)
adj_matrix_val = np.repeat(adj_matrix[np.newaxis, :, :], val_reshaped.shape[0], axis=0)
adj_matrix_test = np.repeat(adj_matrix[np.newaxis, :, :], test_reshaped.shape[0], axis=0)

ValueError: axes don't match array

In [ ]:


# Create Model 
model = modellib.gcn.create_gcn_model(NUM_NODES, NUM_FEATURES)
model.compile(
    optimizer='adam',
    loss=lambda y_true, y_pred: modellib.losses.weighted_binary_crossentropy(y_true, y_pred, class_weights),
    metrics=[
        'accuracy',
        'recall'
    ]
)

history = model.fit(
    [train_reshaped, adj_matrix_train],
    train_labels,
    epochs=100,
    batch_size=32,
    validation_data=([val_reshaped, adj_matrix_val], val_labels),
    class_weight=class_weights
)

# Make predictions on the test set
y_pred_proba = model.predict([test_reshaped, adj_matrix_test])
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions


# Generate Data



In [ ]:
def reshape_bands(matrixbands):
    return np.squeeze(matrixbands).transpose(0, 2, 1)

bands_reshaped = reshape_bands(bands)
print(f"Reshaped bands shape: {bands_reshaped.shape}")

def create_initial_adj_matrix(num_nodes, window_size=3):
    adj_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(max(0, i-window_size), min(num_nodes, i+window_size+1)):
            adj_matrix[i, j] = 1
    return adj_matrix

num_nodes = MATRIX_DIM
num_features = DIAGONAL_BAND_RADIUS * 2 + 1

# Create and normalize adjacency matrix
adj_matrix = create_initial_adj_matrix(num_nodes)
adj_matrix = adj_matrix / np.sum(adj_matrix, axis=1, keepdims=True)

# Split the data into train, validation, and test sets
X_train = bands_reshaped[:train_size]
X_val = bands_reshaped[train_size:train_size + val_size]
X_test = bands_reshaped[train_size + val_size:]
y_train = labels[:train_size]
y_val = labels[train_size:train_size + val_size]
y_test = labels[train_size + val_size:]

# Repeat adj_matrix for each sample in train, val, and test sets
adj_matrix_train = np.repeat(adj_matrix[np.newaxis, :, :], X_train.shape[0], axis=0)
adj_matrix_val = np.repeat(adj_matrix[np.newaxis, :, :], X_val.shape[0], axis=0)
adj_matrix_test = np.repeat(adj_matrix[np.newaxis, :, :], X_test.shape[0], axis=0)


class GraphConv(layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            initializer='glorot_uniform',
            name='kernel')
        self.b = self.add_weight(
            shape=(self.units,), initializer='zeros', name='bias')

    def call(self, inputs):
        x, a = inputs
        h = tf.matmul(x, self.w)
        h = tf.matmul(a, h)
        out = h + self.b
        return self.activation(out) if self.activation is not None else out

def create_gcn_model(num_nodes, num_features):
    x_input = Input(shape=(num_nodes, num_features))
    a_input = Input(shape=(num_nodes, num_nodes))

    gc1 = GraphConv(64, activation='relu')([x_input, a_input])
    gc2 = GraphConv(32, activation='relu')([gc1, a_input])
    gc3 = GraphConv(1)([gc2, a_input])

    output = layers.Flatten()(gc3)
    output = layers.Dense(num_nodes, activation='sigmoid')(output)

    model = Model(inputs=[x_input, a_input], outputs=output)
    return model

model = create_gcn_model(num_nodes, num_features)
# Compile the model with appropriate loss and metrics
model.compile(
    optimizer='adam',
    loss=BinaryCrossentropy(),
    metrics=[
        'accuracy',
        'recall'
    ]
)
from tensorflow.keras.utils import compute_class_weight
import tensorflow as tf



# Use the class weights in model.fit()
history = model.fit(
    [X_train, adj_matrix_train],
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=([X_val, adj_matrix_val], y_val),
    class_weight=class_weight_dict
)

# Make predictions on the test set
y_pred_proba = model.predict([X_test, adj_matrix_test])
y_pred = (y_pred_proba > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='samples', zero_division=0)
recall = recall_score(y_test, y_pred, average='samples', zero_division=0)
f1 = f1_score(y_test, y_pred, average='samples', zero_division=0)

# Calculate confusion matrix for each label
mcm = multilabel_confusion_matrix(y_test, y_pred)

# Calculate overall TP and FN
tp = np.sum([cm[1, 1] for cm in mcm])
fn = np.sum([cm[1, 0] for cm in mcm])

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"True Positives: {tp}")
print(f"False Negatives: {fn}")